## Create/Get Workspace

In [ ]:
from azureml.core import Workspace
from azureml.exceptions import UserErrorException, ProjectSystemException
import json

try:
    ws = Workspace.from_config()
    print("Workspace Loaded")

except (UserErrorException, ProjectSystemException) as e:
    with open(".../ws-details.json") as f:
        ws_details = json.load(f)

    workspace_name = ws_details["workspace_name"]
    subscription_id = ws_details["subscription_id"]
    resource_group = ws_details["resource_group"]
    workspace_region = ws_details["workspace_region"]

    ws = Workspace.create(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        location=workspace_region,
        create_resource_group=True,
        exist_ok=True,
    )

    ws.write_config()
    print("Workspace Created")

## Create/Get Compute Cluster

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Name your cluster
compute_name = "gen-purpose"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("Found compute target: " + compute_name)
else:
    print("Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D16S_V3", max_nodes=10
    )
    # Create the compute target
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())